# OpenAI, Azure OpenAI or compatible APIs

More info about using OpenAI, Azure OpenAI models or compatible APIs with `outlines` [here](https://dottxt-ai.github.io/outlines/latest/reference/models/openai/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [ ]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.openai import OpenAIJSONGenerator

# or for Azure
# from outlines_haystack.generators.azure_openai import AzureOpenAIJSONtGenerator

In [ ]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [ ]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [ ]:
generator = OpenAIJSONGenerator(model_name="gpt-4o-mini", schema_object=Order, generation_kwargs={"temperature": 0.5})

### Standalone

In [ ]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt)

### In a Pipeline

In [ ]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=OpenAIJSONGenerator(
        model_name="gpt-4o-mini",
        schema_object=Order,
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run({"prompt_builder": {"order": "Is it possible to have 12 margheritas?"}})

## Choice Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.openai import OpenAIChoiceGenerator

# or for Azure
# from outlines_haystack.generators.azure_openai import AzureOpenAIChoiceGenerator

In [ ]:
generator = OpenAIChoiceGenerator(
    model_name="gpt-4o-mini",
    choices=["Positive", "Negative"],
    generation_kwargs={"temperature": 0.5},
)

### Standalone

In [ ]:
generator.run(prompt="Classify the following statement: 'I love pizza'")

### In a Pipeline

In [ ]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=OpenAIChoiceGenerator(
        model_name="gpt-4o-mini",
        choices=["Positive", "Negative"],
        generation_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run({"prompt_builder": {"statement": "I love Italian food"}})

## Text Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.openai import OpenAITextGenerator

# or for Azure
# from outlines_haystack.generators.azure_openai import AzureOpenAITextGenerator

In [ ]:
generator = OpenAITextGenerator(model_name="gpt-4o-mini", generation_kwargs={"temperature": 0.5})

### Standalone

In [ ]:
generator.run(prompt="What is the capital of Italy?")

### In a Pipeline

In [ ]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=OpenAITextGenerator(model_name="gpt-4o-mini", generation_kwargs={"temperature": 0.5}),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

In [ ]:
pipeline.run({"prompt_builder": {"country": "France"}})